# roberta finetune on squad

In [21]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management

In [1]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=1027654f8587e407896c392816653755fea39b4fd7a8e06853d54fe662110268
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 156.4 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [22]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [23]:
%%time
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 7274 (delta 9), reused 6 (delta 0), pack-reused 7255
Receiving objects: 100% (7274/7274), 13.87 MiB | 20.95 MiB/s, done.
Resolving deltas: 100% (4909/4909), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-szk4kcrj
Created temporary directory: /tmp/pip-req-tracker-ot2zihz2
Created requirements tracker '/tmp/pip-req-tracker-ot2zihz2'
Created temporary directory: /tmp/pip-install-m85t2hjp
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-cehjljcp
  Added file:///content/transformers/examples/question-answering/apex to build tracker '/tmp/pip-req-tracke

In [24]:
import torch
print(torch.__version__)

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

1.5.0+cu101
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [25]:
!pip install transformers

In [26]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import time
import timeit

In [27]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 31778 (delta 4), reused 10 (delta 4), pack-reused 31763
Receiving objects: 100% (31778/31778), 31.61 MiB | 25.41 MiB/s, done.
Resolving deltas: 100% (22074/22074), done.


In [28]:
from google.colab import auth
auth.authenticate_user()

In [29]:
BUCKET = 'cto-rnd-deep-autocomplete-bucket1' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'roberta_output' #@param {type:"string"}
data_dir_name = 'roberta_data_dir' #@param {type:"string"}
cache_dir_name = 'roberta_cache_dir' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
DATA_DIR = 'gs://{}/{}'.format(BUCKET,data_dir_name)
CACHE_DIR = 'gs://{}/{}'.format(BUCKET,cache_dir_name)
###mkdir(OUTPUT_DIR)
print(f'# Model OUTPUT directory: "{OUTPUT_DIR}"')
print(f'# Model DATA directory: "{DATA_DIR}"')
print(f'# Model CACHE directory: "{CACHE_DIR}"')

# Model OUTPUT directory: "gs://cto-rnd-deep-autocomplete-bucket1/roberta_output"
# Model DATA directory: "gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir"
# Model CACHE directory: "gs://cto-rnd-deep-autocomplete-bucket1/roberta_cache_dir"


In [30]:
!gsutil ls -l $DATA_DIR

        11  2020-06-18T12:05:40Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/
2746816772  2020-06-18T16:33:21Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/cached_train_roberta-base_384
 103598419  2020-06-18T16:32:53Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/cached_train_roberta-base_384.7z
   4854279  2020-06-18T13:41:11Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/dev-v1.1.json
   4370528  2020-06-18T12:10:34Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/dev-v2.0.json
       366  2020-06-18T13:28:27Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/tensor.pt
       364  2020-06-18T13:32:47Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/tensor_x.pt
  30288272  2020-06-18T13:41:12Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/train-v1.1.json
  42123633  2020-06-18T12:10:31Z  gs://cto-rnd-deep-autocomplete-bucket1/roberta_data_dir/train-v2.0.json
TOTAL: 9 objects, 2932052644 bytes (2.73 GiB

In [31]:
###Download the SQUAD train and dev dataset
if 0:
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
  !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [32]:
if 0:
  !gsutil cp /content/train-v2.0.json $DATA_DIR
  !gsutil cp /content/dev-v2.0.json $DATA_DIR

In [33]:
mkdir /data

mkdir: cannot create directory ‘/data’: File exists


In [22]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  19787      0 --:--:-- --:--:-- --:--:-- 19787
OK
50 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 50 not upgraded.
Need to get 4,274 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.28.1_amd64.deb ...
Unpacking gcsfuse (0.28.1) ...
Setting up gcsfuse (0.28.1) ...


In [23]:
!gcsfuse $BUCKET /data

Using mount point: /data
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


In [34]:
!ls -l /data/

total 0
drwxr-xr-x 1 root root 0 Jun 18 12:59 data0519
drwxr-xr-x 1 root root 0 Jun 18 12:59 qa
drwxr-xr-x 1 root root 0 Jun 18 16:45 roberta_cache_dir
drwxr-xr-x 1 root root 0 Jun 18 12:28 roberta_data_dir
drwxr-xr-x 1 root root 0 Jun 18 12:59 roberta_output


In [35]:
OUTPUT_DIR = '/data/roberta_output'
DATA_DIR = '/data/roberta_data_dir'
CACHE_DIR = '/data/roberta_cache_dir'

In [36]:
!ls -l $DATA_DIR

total 2863335
-rw-r--r-- 1 root root 2746816772 Jun 18 16:33 cached_train_roberta-base_384
-rw-r--r-- 1 root root  103598419 Jun 18 16:32 cached_train_roberta-base_384.7z
-rw-r--r-- 1 root root    4854279 Jun 18 13:41 dev-v1.1.json
-rw-r--r-- 1 root root    4370528 Jun 18 12:10 dev-v2.0.json
-rw-r--r-- 1 root root        366 Jun 18 13:28 tensor.pt
-rw-r--r-- 1 root root        364 Jun 18 13:32 tensor_x.pt
-rw-r--r-- 1 root root   30288272 Jun 18 13:41 train-v1.1.json
-rw-r--r-- 1 root root   42123633 Jun 18 12:10 train-v2.0.json


In [37]:
cd transformers/examples/question-answering

/content/transformers/examples/question-answering/transformers/examples/question-answering


In [38]:
!ls -l

total 56
-rw-r--r-- 1 root root  5022 Jun 18 17:18 README.md
-rw-r--r-- 1 root root 34349 Jun 18 17:18 run_squad.py
-rw-r--r-- 1 root root  8757 Jun 18 17:18 run_tf_squad.py


In [ ]:
!python run_squad.py \
        --model_type roberta \
        --model_name_or_path roberta-base \
        --do_train \
        --do_eval \
        --per_gpu_train_batch_size 16 \
        --gradient_accumulation_steps 12 \
        --max_grad_norm 1 \
        --learning_rate 3e-5 \
        --warmup_steps 0 \
        --num_train_epochs 6 \
        --max_seq_length 384 \
        --max_query_length 64 \
        --max_answer_length 30 \
        --doc_stride 128 \
        --version_2_with_negative \
        --do_lower_case \
        --save_steps 400 \
        --seed 42 \
        --train_file train-v2.0.json \
        --predict_file dev-v2.0.json \
        --output_dir $OUTPUT_DIR \
        --data_dir  $DATA_DIR \
        --cache_dir  $CACHE_DIR \
        --overwrite_output_dir \
        --fp16

2020-06-18 17:20:00.649016: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
06/18/2020 17:20:07 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
06/18/2020 17:20:07 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /data/roberta_cache_dir/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
06/18/2020 17:20:07 - INFO - transformers.configuration_utils -   Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_ep